# suraksha **Ai**

importing libraries

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

**loading the data sheet**

In [2]:
df = pd.read_csv('/content/crime_in_la.csv', on_bad_lines='skip', engine='python')
df

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,Vict Sex,Vict Descent,Premis Desc,Weapon Used Cd,Weapon Desc,Status,Status Desc,LOCATION,LAT,LON
0,190326475,01/03/2020,21:30,7,Wilshire,784,510,VEHICLE - STOLEN,0,0,M,O,STREET,500.0,UNKNOWN WEAPON/OTHER WEAPON,AA,Adult Arrest,1900 S LONGWOOD AV,34.0375,-118.3506
1,200106753,08/02/2020,18:00,1,Central,182,330,BURGLARY FROM VEHICLE,1822 1402 0344,47,M,O,BUS STOP/LAYOVER (ALSO QUERY 124),500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1000 S FLOWER ST,34.0444,-118.2628
2,200320258,04/11/2020,17:00,3,Southwest,356,480,BIKE - STOLEN,0344 1251,19,X,X,"MULTI-UNIT DWELLING (APARTMENT, DUPLEX, ETC)",500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1400 W 37TH ST,34.0210,-118.3002
3,200907217,10/03/2020,20:37,9,Van Nuys,964,343,SHOPLIFTING-GRAND THEFT ($950.01 & OVER),0325 1501,19,M,O,CLOTHING STORE,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,14000 RIVERSIDE DR,34.1576,-118.4387
4,220614831,17/08/2020,12:00,6,Hollywood,666,354,THEFT OF IDENTITY,1822 1501 0930 2004,28,M,H,SIDEWALK,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1900 TRANSIENT,34.0944,-118.3277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963001,242106032,17/02/2024,23:30,21,Topanga,2103,420,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),0385,44,F,O,DRIVEWAY,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,22700 NAPA ST,34.2260,-118.6214
963002,242004546,16/01/2024,15:10,20,Olympic,2013,624,BATTERY - SIMPLE ASSAULT,2047 0400 0429 0444 1202 1822,80,F,O,BUS STOP,400.0,"STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)",IC,Invest Cont,HOBART BL,34.0690,-118.3054
963003,240104953,15/01/2024,01:00,1,Central,101,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 0400 0416,0,X,X,HOTEL,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,1300 W SUNSET BL,34.0685,-118.2460
963004,241711348,19/07/2024,07:57,17,Devonshire,1751,888,TRESPASSING,1501,0,X,X,MTA - ORANGE LINE - CHATSWORTH,500.0,UNKNOWN WEAPON/OTHER WEAPON,IC,Invest Cont,10000 OLD DEPOT PLAZA RD,34.2500,-118.5990


**Data preprocessing**

In [3]:
df.columns

Index(['DR_NO', 'DATE OCC', 'TIME OCC', 'AREA', 'AREA NAME', 'Rpt Dist No',
       'Crm Cd', 'Crm Cd Desc', 'Mocodes', 'Vict Age', 'Vict Sex',
       'Vict Descent', 'Premis Desc', 'Weapon Used Cd', 'Weapon Desc',
       'Status', 'Status Desc', 'LOCATION', 'LAT', 'LON'],
      dtype='object')

In [4]:
df = df[['LAT', 'LON', 'TIME OCC', 'Crm Cd Desc']]
df = df.dropna()

In [5]:
df['TIME OCC'] = pd.to_numeric(df['TIME OCC'], errors='coerce')
df['HOUR'] = df['TIME OCC'] // 100
df.drop(columns=['TIME OCC'], inplace=True)

**Fixing the Threshold**

In [6]:
def danger_level(crime):
    crime = crime.lower()
    if 'rape' in crime or 'homicide' in crime or 'assault with deadly weapon' in crime or 'kidnap' in crime:
        return 2   # Dangerous
    elif 'robbery' in crime or 'assault' in crime or 'sexual' in crime:
        return 1   # Not Safe
    else:
        return 0   # Safe

df['danger'] = df['Crm Cd Desc'].apply(danger_level)
df.drop(columns=['Crm Cd Desc'], inplace=True)

In [7]:
X = df[['LAT', 'LON', 'HOUR']]
y = df['danger']

**spliting the datasheet into train and sheet**

In [8]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

**Importing the pre-trained model**

In [12]:
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

**Testing the Accuracy**

In [13]:
y_pred = model.predict(X_test)
accuracy_score(y_test, y_pred)

0.7314098503650014

In [14]:
# Example: Night time location
sample = [[34.0407, -118.2468, 19]]  # LAT, LON, HOUR
prediction = model.predict(sample)

labels = ['Safe', 'Not Safe', 'Dangerous']
labels[prediction[0]]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Not Safe'

In [15]:
# Example: Night time location
sample = [[34.05, -118.25, 22]]  # LAT, LON, HOUR
prediction = model.predict(sample)

labels = ['Safe', 'Not Safe', 'Dangerous']
labels[prediction[0]]

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


'Safe'